In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf


In [3]:
#setting up to load our dataset
from google.colab import files
uploaded = files.upload()
#choose processed_student_performance.csv

Saving processed_student_performance.csv to processed_student_performance.csv


In [4]:
#import our proccessed dataset
processed_df = pd.read_csv('processed_student_performance.csv')
processed_df.head()

,gender_female,gender_male,race/ethnicity_group A,race/ethnicity_group B,race/ethnicity_group C,race/ethnicity_group D,race/ethnicity_group E,parental level of education_associate's degree,parental level of education_bachelor's degree,parental level of education_high school,parental level of education_master's degree,parental level of education_some college,parental level of education_some high school,lunch_free/reduced,lunch_standard,test preparation course_completed,test preparation course_none,average score,success
0,True,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,True,0.325171,False
1,True,False,False,False,True,False,False,False,False,False,False,True,False,False,True,True,False,0.977607,False
2,True,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,True,1.663432,False
3,False,True,True,False,False,False,False,True,False,False,False,False,False,True,False,False,True,-1.225729,False
4,False,True,False,False,True,False,False,False,False,False,False,True,False,False,True,False,True,0.572148,False


In [5]:
processed_df.columns

Index(['gender_female', 'gender_male', 'race/ethnicity_group A',
       'race/ethnicity_group B', 'race/ethnicity_group C',
       'race/ethnicity_group D', 'race/ethnicity_group E',
       'parental level of education_associate's degree',
       'parental level of education_bachelor's degree',
       'parental level of education_high school',
       'parental level of education_master's degree',
       'parental level of education_some college',
       'parental level of education_some high school', 'lunch_free/reduced',
       'lunch_standard', 'test preparation course_completed',
       'test preparation course_none', 'average score', 'success'],
      dtype='object')

In [15]:
features= processed_df[['gender_female', 'gender_male',
                        'race/ethnicity_group A', 'race/ethnicity_group B', 'race/ethnicity_group C', 'race/ethnicity_group D', 'race/ethnicity_group E',
                        "parental level of education_associate's degree", "parental level of education_bachelor's degree", 'parental level of education_high school',
                        "parental level of education_master's degree", 'parental level of education_some college', 'parental level of education_some high school',
                        'lunch_free/reduced','lunch_standard',
                        'test preparation course_completed','test preparation course_none']]
target = processed_df['success']

In [16]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [23]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features_total = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = input_features_total, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 8)                   │             144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │              45 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │               6 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 195 (780.00 B)

 Trainable params: 195 (780.00 B)

 Non-trainable params: 0 (0.00 B)

In [28]:
# Compile the model (FIRST ATTEMPT)
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Train the model (FIRST ATTEMPT)
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5274 - loss: 0.7739
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6898 - loss: 0.7101
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8496 - loss: 0.6433
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9324 - loss: 0.5820
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9837 - loss: 0.5094
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9964 - loss: 0.4315
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.3509 
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.2777 
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.2091
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.1610 
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.1214 
Epoch 12/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accur

In [30]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

7/7 - 0s - 36ms/step - accuracy: 1.0000 - loss: 2.4315e-04
Loss: 0.0002431546017760411, Accuracy: 1.0


## Tests for other algorithms

In [24]:
from sklearn.ensemble import RandomForestRegressor

# Choose and train the model
model = RandomForestRegressor()
model.fit(X_train, y_train)


RandomForestRegressor()

In [25]:
from sklearn.metrics import mean_squared_error, r2_score

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')


Mean Squared Error: 0.0
R^2 Score: 1.0


In [ ]:
#why does it somehow work now??

## Utilizing Spark

In [35]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,118 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [

In [36]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


In [38]:
#read in data
df = spark.read.csv("StudentsPerformance.csv", header=True, inferSchema=True)

# Show DataFrame
df.show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     

In [40]:
#create a tempview
df.createOrReplaceTempView('performance')

In [54]:
spark.sql("""
  SELECT
    gender,
    count(gender)
  FROM performance
  GROUP BY gender
  """).show()

+------+-------------+
|gender|count(gender)|
+------+-------------+
|female|          518|
|  male|          482|
+------+-------------+



In [65]:
spark.sql("""
  SELECT
    `race/ethnicity`,
    COUNT(*) AS count
  FROM performance
  GROUP BY `race/ethnicity`
  ORDER BY count DESC
""").show()

+--------------+-----+
|race/ethnicity|count|
+--------------+-----+
|       group C|  319|
|       group D|  262|
|       group B|  190|
|       group E|  140|
|       group A|   89|
+--------------+-----+



In [64]:
spark.sql("""
  SELECT
    `parental level of education`,
    COUNT(*) AS count
  FROM performance
  GROUP BY `parental level of education`
  ORDER BY count DESC
""").show()

+---------------------------+-----+
|parental level of education|count|
+---------------------------+-----+
|               some college|  226|
|         associate's degree|  222|
|                high school|  196|
|           some high school|  179|
|          bachelor's degree|  118|
|            master's degree|   59|
+---------------------------+-----+



In [66]:
spark.sql("""
  SELECT
    `lunch`,
    COUNT(*) AS count
  FROM performance
  GROUP BY `lunch`
  ORDER BY count DESC
""").show()

+------------+-----+
|       lunch|count|
+------------+-----+
|    standard|  645|
|free/reduced|  355|
+------------+-----+



In [67]:
spark.sql("""
  SELECT
    `test preparation course`,
    COUNT(*) AS count
  FROM performance
  GROUP BY `test preparation course`
  ORDER BY count DESC
""").show()

+-----------------------+-----+
|test preparation course|count|
+-----------------------+-----+
|                   none|  642|
|              completed|  358|
+-----------------------+-----+



In [81]:
spark.sql("""
  SELECT
    `parental level of education`,
    AVG(`math score`),
    AVG(`reading score`),
    AVG(`writing score`),
    GREATEST(AVG(`math score`), AVG(`reading score`), AVG(`writing score`)) AS max_avg_score
  FROM performance
  GROUP BY `parental level of education`
  ORDER BY max_avg_score DESC
""").show()

+---------------------------+------------------+------------------+------------------+-----------------+
|parental level of education|   avg(math score)|avg(reading score)|avg(writing score)|    max_avg_score|
+---------------------------+------------------+------------------+------------------+-----------------+
|            master's degree|  69.7457627118644| 75.37288135593221| 75.67796610169492|75.67796610169492|
|          bachelor's degree| 69.38983050847457|              73.0| 73.38135593220339|73.38135593220339|
|         associate's degree| 67.88288288288288| 70.92792792792793|  69.8963963963964|70.92792792792793|
|               some college|  67.1283185840708| 69.46017699115045| 68.84070796460178|69.46017699115045|
|           some high school|63.497206703910614| 66.93854748603351| 64.88826815642459|66.93854748603351|
|                high school| 62.13775510204081| 64.70408163265306| 62.44897959183673|64.70408163265306|
+---------------------------+------------------+-------

In [80]:
spark.sql("""
  SELECT
    lunch,
    AVG(`math score`),
    AVG(`reading score`),
    AVG(`writing score`),
    GREATEST(AVG(`math score`), AVG(`reading score`), AVG(`writing score`)) AS max_avg_score
  FROM performance
  GROUP BY lunch
  ORDER BY max_avg_score DESC
""").show()

+------------+-----------------+------------------+------------------+-----------------+
|       lunch|  avg(math score)|avg(reading score)|avg(writing score)|    max_avg_score|
+------------+-----------------+------------------+------------------+-----------------+
|    standard|70.03410852713178| 71.65426356589147|  70.8232558139535|71.65426356589147|
|free/reduced|58.92112676056338| 64.65352112676057| 63.02253521126761|64.65352112676057|
+------------+-----------------+------------------+------------------+-----------------+



In [79]:
spark.sql("""
  SELECT
    `test preparation course`,
    AVG(`math score`),
    AVG(`reading score`),
    AVG(`writing score`),
    GREATEST(AVG(`math score`), AVG(`reading score`), AVG(`writing score`)) AS max_avg_score
  FROM performance
  GROUP BY `test preparation course`
  ORDER BY avg(`math score`) DESC
""").show()

+-----------------------+-----------------+------------------+------------------+-----------------+
|test preparation course|  avg(math score)|avg(reading score)|avg(writing score)|    max_avg_score|
+-----------------------+-----------------+------------------+------------------+-----------------+
|              completed|69.69553072625699| 73.89385474860335| 74.41899441340782|74.41899441340782|
|                   none| 64.0778816199377| 66.53426791277259| 64.50467289719626|66.53426791277259|
+-----------------------+-----------------+------------------+------------------+-----------------+



In [78]:
spark.sql("""
  SELECT
    gender,
    AVG(`math score`),
    AVG(`reading score`),
    AVG(`writing score`),
    GREATEST(AVG(`math score`), AVG(`reading score`), AVG(`writing score`)) AS max_avg_score
  FROM performance
  GROUP BY gender
  ORDEr BY max_avg_score DESC
""").show()

+------+------------------+------------------+------------------+-----------------+
|gender|   avg(math score)|avg(reading score)|avg(writing score)|    max_avg_score|
+------+------------------+------------------+------------------+-----------------+
|female|63.633204633204635| 72.60810810810811| 72.46718146718146|72.60810810810811|
|  male| 68.72821576763485| 65.47302904564316| 63.31120331950208|68.72821576763485|
+------+------------------+------------------+------------------+-----------------+



In [86]:
spark.sql("""
  SELECT
    `race/ethnicity`,
    AVG(`math score`),
    AVG(`reading score`),
    AVG(`writing score`),
    GREATEST(AVG(`math score`), AVG(`reading score`), AVG(`writing score`)) AS max_avg_score
  FROM performance
  GROUP BY `race/ethnicity`
  ORDER BY avg(`math score`) DESC
""").show()

+--------------+-----------------+------------------+------------------+-----------------+
|race/ethnicity|  avg(math score)|avg(reading score)|avg(writing score)|    max_avg_score|
+--------------+-----------------+------------------+------------------+-----------------+
|       group E|73.82142857142857| 73.02857142857142| 71.40714285714286|73.82142857142857|
|       group D|67.36259541984732| 70.03053435114504| 70.14503816793894|70.14503816793894|
|       group C|64.46394984326018| 69.10344827586206| 67.82758620689656|69.10344827586206|
|       group B|63.45263157894737| 67.35263157894737|              65.6|67.35263157894737|
|       group A|61.62921348314607| 64.67415730337079|62.674157303370784|64.67415730337079|
+--------------+-----------------+------------------+------------------+-----------------+

